<a href="https://colab.research.google.com/github/erchivox/SIEMFAP-Sistema-Inteligente-de-Ecoturismo-y-Monitoreo-de-Fauna-en-reas-Protegidas-/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

estamos actualizando la base de datos para mejorar el modelo

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


#carga del modelo VGG16 preentrenado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

#agregar capas personalizadas encima del modelo base
x = base_model.output
x = Flatten()(x)  # Aplanar la salida
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  #apagar el 50% de las neuronas y evitar sobreajuste
predictions = Dense( , activation='softmax')(x)  #estamos desidiendo las categorias que tendra

# Definir el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Crear generadores de datos para entrenamiento y validación
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Definir las rutas a tus carpetas de entrenamiento y validación (por definir)
train_dir = ''
validation_dir = ''

# Generador para el conjunto de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode="rgb")

# Generador para el conjunto de validación
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode="rgb")

print("Clases en entrenamiento:", train_generator.class_indices)
print("Clases en validación:", validation_generator.class_indices)

# entrenamiento de la ultima capa
model.fit(
    train_generator,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    validation_data=validation_generator,
    validation_steps=max(1, validation_generator.samples // validation_generator.batch_size),
    epochs=20)

# Guardar el modelo entrenado
drive.mount('/content/drive')
model.save('/content/drive/MyDrive/Modelos/modelo_clasificador_vgg16_v1.h5')
